In [1]:
import pandas as pd

Load and Understand Data

In [3]:
df1 = pd.read_csv("bank_transaction.csv")
df2 = pd.read_csv("user_profile.csv")

print(df1.head().to_string())
print(df2.head().to_string())

   client_id  bank_id  account_id  txn_id             txn_date                                                                                                                       description  amount         category
0          1        1           1       4  2023-09-29 00:00:00                                                                           Earnin           PAYMENT                 Donatas Danyal    20.0            Loans
1          1        1           1       3  2023-08-14 00:00:00                                                       ONLINE TRANSFER FROM NDonatas DanyalDA O CARSON BUSINESS CHECKING 1216 1216    25.0  Transfer Credit
2          1        1           1       5  2023-09-25 00:00:00  MONEY TRANSFER                          AUTHORIZED ON   09/25 FROM Earnin CDAEJ_B                     CA  S583269001208168   111    20.0            Loans
3          1        1           2       1  2023-06-02 00:00:00                                                                  